# Plotting 2: Bar plots and scatters

files needed = None

Let's work through a few more of matplotlib's basic figures. We will come back to figures a later to work on some of the more complex visualizations. 

### The packages

In [ ]:
import pandas as pd     #load the pandas package and call it pd
import matplotlib.pyplot as plt   # load the pyplot set of tools from the package matplotlib. Name it plt for short.
from pandas_datareader import data, wb    # we are grabbing the data and wb functions from the package
import datetime as dt                     # for time and date

# This following is a jupyter magic command. It tells jupyter to insert the plots into the notebook
# rather than a new window.
%matplotlib inline      

## Bar charts
Bar charts are useful for describing relatively few observations of categorical data --- meaning that one of the axes is not quantitative. [Tufte](https://www.edwardtufte.com/tufte) would complain that they have a lot of redundant ink, but they are quite popular...and Tufte is not our dictator. Although, it's always good to think about what our figures are doing for us.   

Bar charts are much better than pie charts for displaying the relative size of data. There are discussions of this all over the net (here is [one](http://www.storytellingwithdata.com/blog/2011/07/death-to-pie-charts) I like) but the anti-pie-chart argument boils down to: pie charts are hard to read.   
1. Humans are bad at judging the relative sizes of 2D spaces. They cannot tell if one slice is 10% larger than another slice.
2. The MS Excel style of coloring the slice different colors creates problems. Humans judge darker colors to have larger areas. 
2. To get quantitative traction, people label the slices with the data values. In this case, a table of numbers is probably a better way to share the data.

In [ ]:
# PPP GDP data from the penn world tables 

code    = ['USA', 'FRA', 'JPN', 'CHN', 'IND', 'BRA', 'MEX']
country = ['United States', 'France', 'Japan', 'China', 'India',
             'Brazil', 'Mexico']
gdppc   = [53.1, 36.9, 36.3, 11.9, 5.4, 15.0, 16.5]

gdp = pd.DataFrame({'gdppc': gdppc, 'country': country}, index=code)
gdp

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.bar(gdp.index, gdp['gdppc'], color='blue', alpha=0.25)      # bar(x labels, )

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_ylabel('PPP GDP per capita')
ax.set_title('Income per person (at purchasing power parity)')

plt.show()

The ordering of the bars is pretty random. We could sort it poor to rich.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

gdp_sort= gdp.sort_values('gdppc')

ax.bar(gdp_sort.index, gdp_sort['gdppc'], color='blue', alpha=0.25)      # bar(x labels, )
ax.grid(axis='y', color='white')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_title('Income per person (at purchasing power parity)')
ax.set_ylabel('PPP GDP per capita')

plt.show()

Notice the use of `grid()` to specify grid lines on the y axis. I made them white, so they only show up in the bars. It's something I'm experimenting with. I'm not sure I like it. 

Maybe you prefer a horizontal bar chart. Same data, same approach. We need to swap all the y labels for x labels. 

### Practice: Bar charts
Take a few minutes and try the following. Feel free to chat with those around if you get stuck. The TA and I are here, too.

1. Create a horizontal bar chart of the gdp data we used above. Check the documentation for `barh()`
2. Fix up your figure labels, etc. 

3. Create a new horizontal bar chart where each bar is gdp per capita relative to the United States. So USA =1, MEX = 0.31, etc. 

## Scatter plots

Scatter plots are used to compare two variables. A very common approach to visualize the correlation of two variables. 

In [ ]:
codes = ['GDPC1', 'UNRATE']                                         # real gdp, unemployment rate
start = dt.datetime(1970, 1, 1)
fred = data.DataReader(codes, 'fred', start)

fred.head()


Gremlins! The gdp data is quarterly, but the unemployment rate is monthly. Let's fix this by downsampling to quarterly frequency. The FRED datareader is really good --- the index is already a datetime object. (How would you check?)

In [ ]:
fred_q=fred.resample('q').mean()                # Create an average quarterly unemployment rate
fred_q.head()

Let's plot the growth rate of GDP against the change in the unemployment rate. The relationship between these two variables is known as [Okun's Law](https://en.wikipedia.org/wiki/Okun%27s_law). 

Since the unemployment rates are already rates, it makes more sense to just difference them than to compute growth rates.

Example: $u_t=1.25\%$ and $u_{t-1}=0.75\%$. 
It is clearer to say that the unemployment rate rose by 0.5 percentage points than by 67\% 


In [ ]:
fred_q['gdp_gr'] = fred_q['GDPC1'].pct_change()*100        # growth rate of gdp. we've seen this a few times...
fred_q['unemp_dif'] = fred_q['UNRATE'].diff()              # difference takes the first difference: u(t)-u(t-1)   
fred_q.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
                       
ax.scatter(fred_q.gdp_gr, fred_q.unemp_dif)

ax.set_title('Okun\'s Law in the United States' )
ax.set_ylabel('change in unemployment rate')
ax.set_xlabel('gdp growth rate')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


### Practice: Scatters
Take a few minutes and try the following. Feel free to chat with those around if you get stuck. 

Let's explore some of scatter plot's options. Modify the Okun's Law figure we just made. 

1. Change the color of the dots to red and lighten them up using 'alpha'

Check out the documentation for [marker styles](https://matplotlib.org/api/markers_api.html). 

3. Change the marker to a triangle. 
4. Use `text()` or `annotate()` to label the point corresponding to third quarter 2009: '2009Q3'

Scatter plots are very useful and we can do a lot more with them. Places to go from here.
1. Add a line of best fit. A bit clunky in matplotlib (use np's polyfit command), but not too bad. The *Seaborn* package has a `regplot` command that makes this dead simple. 
2. Make data markers different colors or sizes depending on the value of a third variable. For example, you could get some more data and color the markers for years with a repbulican president red and markers for years with democratic presidents blue. 
3. Other ideas? Let me know!
